In [1]:
import open3d as o3d
import cl
import utils as ut
import numpy as np
from skimage.morphology import binary_dilation
import proc3d
import json
from PIL import Image
from utils import *
import glob
import os
import json

In [2]:
class space_carving_2():
    def __init__(self, dataset_path):
        self.masks_files = sorted (glob.glob(os.path.join(dataset_path, 'masks', '*.png')) )#get all .png file names from folder path
        self.extrinsics = self.load_extrinsics(os.path.join(dataset_path, 'extrinsics'))
        self.bbox = json.load(open(os.path.join(dataset_path, 'bbox.json')))
        self.camera_model = json.load(open(os.path.join(dataset_path, 'camera_model.json')))
        self.intrinsics= self.camera_model['params'][0:4]
        
        params = json.load(open(os.path.join(dataset_path, 'params.json')))
        self.gt=o3d.io.read_point_cloud(params["gt_path"])
        self.gt_points = np.asarray(self.gt.points)
        self.n_dilation=params["sc"]["n_dilation"]
        self.voxel_size = params['sc']['voxel_size']
        
        self.set_sc(self.bbox)
        
    def reset(self):
        del(self.sc)
        self.set_sc(self.bbox) 
        
    def load_extrinsics(self,path):
        ext = []
        ext_files = glob.glob(os.path.join(path, '*.json'))
        assert len(ext_files) != 0,"json list is empty."
        for i in sorted(ext_files):                                                                                                                                     
            ext.append(json.load(open(i)))                                                                                                                                                                                                                                                  
        return ext 
    
    def load_mask(self,idx):                                                                                                                                         
        img = cv2.imread(self.masks_files[idx], cv2.IMREAD_GRAYSCALE)                                                                                                                                                                                                                                                                                                                                                                                     
        return img

    def set_sc(self,bbox):
        x_min, x_max = bbox['x']
        y_min, y_max = bbox['y']
        z_min, z_max = bbox['z']

        nx = int((x_max - x_min) / self.voxel_size) + 1
        ny = int((y_max - y_min) / self.voxel_size) + 1
        nz = int((z_max - z_min) / self.voxel_size) + 1

        self.origin = np.array([x_min, y_min, z_min])
        self.sc = cl.Backprojection([nx, ny, nz], [x_min, y_min, z_min], self.voxel_size)

    def carve(self,idx):
        im = self.load_mask(idx)
        self.space_carve(im, self.extrinsics[idx])
        
    def space_carve(self, mask, rt):
        #mask = im.copy() #get_mask(im)
        rot = sum(rt['R'], [])
        tvec = rt['T']
        if self.n_dilation:
            for k in range(self.n_dilation): mask = binary_dilation(mask)    
        self.sc.process_view(self.intrinsics, rot, tvec, mask)
        
    def dist_to_gt(self):
        vol = self.sc.values().copy()
        vol = vol.reshape(self.sc.shape)
        pcd=proc3d.vol2pcd_exp(vol, self.origin, self.voxel_size, level_set_value=0) 
        pcd_p = np.asarray(pcd.points)
        cd=chamfer_d(self.gt_points , pcd_p)
        return cd

In [5]:
data_path = '/home/pico/uni/romi/rl_sony/arabidopsis_image_sets/'
#plants = [ '001_2d','003_2d','006_2d','009_2d','124_2d','195_2d']
#plants = ['000_2d']

In [6]:
for plant in plants:
    sc = space_carving_2(os.path.join(data_path,plant))
    min_max = {}
    #carve all images to get min achievable distance
    sc.reset()
    for i in range(720):
        sc.carve(i)
        if i==179: #first row completed
            min_max['1d_min'] = sc.dist_to_gt()
            
    min_max['min'] = sc.dist_to_gt()
    
    #carve each image independently
    distances = []
    for i in range(720):
        sc.reset()
        sc.carve(i)
        dist = sc.dist_to_gt()
        distances.append(dist)
        
        if i==179: #first row completed
            min_max['1d_max'] = max(distances)           
        print(i,f"{dist}       ",end='\r')
        
    min_max['max'] = max(distances)
    
    with open(os.path.join(data_path,plant,'chamfer_distance_gt_limits.json'), 'w') as json_file:
            json.dump(min_max, json_file)